<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'>Required Libraries and Modules</h2>
</div>

**Note: This notebook is run in Paperspace platform**

In [ ]:
!pip install accelerate==0.21.0
!pip install transformers==4.24.0

In [ ]:
# 查看GPU信息
! /opt/bin/nvidia-smi


Thu May 23 15:07:24 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, StratifiedKFold, StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, classification_report

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import EarlyStoppingCallback, set_seed

# Progress bar
from tqdm._tqdm_notebook import tqdm_notebook
from tqdm import tqdm
tqdm_notebook.pandas()

<ipython-input-3-5c70c929f42e>:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  from tqdm._tqdm_notebook import tqdm_notebook


In [ ]:
import pandas as pd

<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'>Import Clean Text Data</h2>
</div>

In [ ]:

# df2 = pd.read_csv('/content/drive/MyDrive/balanced_df_all_cleaned_tokenization.csv')
df2 = pd.read_csv('/content/drive/MyDrive/balanced_df_all_cleaned.csv')

In [ ]:
# choose 7000 rows to retrain the models
df_label_0 = df2[df2['label'] == 0].sample(n=1000, random_state=43)
df_label_1 = df2[df2['label'] == 1].sample(n=1000, random_state=43)

# merge two dataframes
df3 = pd.concat([df_label_0, df_label_1])


# bert base chinese

In [ ]:
!pip install transformers datasets pandas


In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [ ]:
# data =df3 # 2000 rows
data =df2 # all rows

In [ ]:
data.shape

(20650, 3)

In [ ]:
# 确保tweet_content列是字符串
data['tweet_content'] = data['tweet_content'].astype(str)


In [ ]:
# 将DataFrame转换为Hugging Face的Dataset对象
train_data, test_data = train_test_split(data, test_size=0.1, stratify=data['label'])
train_dataset = Dataset.from_pandas(train_data)
test_dataset = Dataset.from_pandas(test_data)

In [ ]:
# 加载预训练的BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# 定义一个tokenizer函数，将tweet_content转换为BERT模型可接受的格式
def tokenize_function(example):
    return tokenizer(example['tweet_content'], padding='max_length', truncation=True)


In [ ]:
# 对数据集进行tokenization
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/18585 [00:00<?, ? examples/s]

Map:   0%|          | 0/2065 [00:00<?, ? examples/s]

In [ ]:
# 定义计算评价指标的函数
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

In [ ]:
# 设置训练参数
training_args = TrainingArguments(
    output_dir='./results',            # 输出目录
    num_train_epochs=8,                # 训练轮数
    per_device_train_batch_size=8,     # 训练时每个设备的批量大小
    per_device_eval_batch_size=8,      # 评估时每个设备的批量大小
    warmup_steps=500,                  # 预热步数
    weight_decay=0.01,                 # 权重衰减
    logging_dir='./logs',              # 日志目录
    logging_steps=10,                  # 日志记录步数
    evaluation_strategy='epoch'        # 每轮结束后评估模型
)

In [ ]:
# 定义Trainer对象
# model_name = "distilbert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained("bert-base-chinese", num_labels=2)
trainer = Trainer(
    model=model,                       # 训练的模型
    args=training_args,                # 训练参数
    train_dataset=train_dataset,       # 训练数据集
    eval_dataset=test_dataset,         # 测试数据集
    compute_metrics=compute_metrics    # 计算评价指标的函数
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-chinese and are newly i

In [ ]:
# 开始训练 all rows, 3 epoch,test size=0.2
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 16520
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 6195
  Number of trainable parameters = 102269186
The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: ID, __index_level_0__, tweet_content. If ID, __index_level_0__, tweet_content are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.122800,0.457974,0.915738,0.890050,0.948668,0.918425
2,0.427900,0.256882,0.930266,0.962038,0.895884,0.927783
3,0.265500,0.251773,0.931719,0.940682,0.921550,0.931018


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Model weights saved in ./results/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-2000
Configuration saved in ./results/checkpoint-2000/config.json
Model weights saved in ./results/checkpoint-2000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 4130
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: ID, __index_level_0__, tweet_content. If ID, __index_level_0__,

TrainOutput(global_step=6195, training_loss=0.28746494348247364, metrics={'train_runtime': 5355.6937, 'train_samples_per_second': 9.254, 'train_steps_per_second': 1.157, 'total_flos': 1.30397839036416e+16, 'train_loss': 0.28746494348247364, 'epoch': 3.0})

In [ ]:
# 开始训练 all rows,8 epoch,test size=0.1
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: ID, tweet_content, __index_level_0__. If ID, tweet_content, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 18585
  Num Epochs = 8
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 18592
  Number of trainable parameters = 102269186


Epoch,Training Loss,Validation Loss


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin


In [ ]:
# 开始训练 2000rows,3 epoch,test size=0.2
trainer.train()



***** Running training *****
  Num examples = 1600
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 600
  Number of trainable parameters = 102269186
The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: ID, __index_level_0__, tweet_content. If ID, __index_level_0__, tweet_content are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.188500,0.262527,0.917500,0.903382,0.935000,0.918919
2,0.070200,0.396343,0.910000,0.886792,0.940000,0.912621
3,0.096400,0.396343,0.910000,0.886792,0.940000,0.912621


***** Running Evaluation *****
  Num examples = 400
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: ID, __index_level_0__, tweet_content. If ID, __index_level_0__, tweet_content are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.


Epoch,Training Loss,Validation Loss


***** Running Evaluation *****
  Num examples = 400
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: ID, __index_level_0__, tweet_content. If ID, __index_level_0__, tweet_content are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 400
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: ID, __index_level_0__, tweet_content. If ID, __index_level_0__, tweet_content are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.


Training completed. Do not forget to s

TrainOutput(global_step=600, training_loss=0.19838822609434525, metrics={'train_runtime': 523.8595, 'train_samples_per_second': 9.163, 'train_steps_per_second': 1.145, 'total_flos': 1262933065728000.0, 'train_loss': 0.19838822609434525, 'epoch': 3.0})

In [ ]:
# 进行评估 all rows，test size=0.1
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

***** Running Evaluation *****
  Num examples = 4130
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: ID, __index_level_0__, tweet_content. If ID, __index_level_0__, tweet_content are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.


Evaluation results: {'eval_loss': 0.25177332758903503, 'eval_accuracy': 0.9317191283292978, 'eval_precision': 0.9406821552150272, 'eval_recall': 0.9215496368038741, 'eval_f1': 0.9310176125244618, 'eval_runtime': 131.5244, 'eval_samples_per_second': 31.401, 'eval_steps_per_second': 3.931, 'epoch': 3.0}


In [ ]:
# 进行评估 2000rows
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

***** Running Evaluation *****
  Num examples = 400
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: ID, __index_level_0__, tweet_content. If ID, __index_level_0__, tweet_content are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.


Evaluation results: {'eval_loss': 0.3963434100151062, 'eval_accuracy': 0.91, 'eval_precision': 0.8867924528301887, 'eval_recall': 0.94, 'eval_f1': 0.912621359223301, 'eval_runtime': 13.1626, 'eval_samples_per_second': 30.389, 'eval_steps_per_second': 3.799, 'epoch': 3.0}
